In [ ]:
%%capture
!pip install tiktoken openai langchain

In [ ]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains import ReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.document_loaders import WebBaseLoader
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
import textwrap

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "secret_key"

In [ ]:
model_name = "gpt-3.5-turbo-16k"
temperature = 0.1

In [ ]:
llm = ChatOpenAI(model_name=model_name, temperature=temperature)

In [ ]:
loader = WebBaseLoader("https://simple.wikipedia.org/wiki/Artificial_intelligence")

def load_folder(folder):
    documents = loader.load()
    return documents

documents = load_folder(loader)
len(documents)

1

In [ ]:
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

docs = split_docs(documents)
print(len(docs))

16


In [ ]:
docs[8]

Document(page_content='AI involves many different fields like computer science, mathematics, linguistics, psychology, neuroscience, and philosophy. Eventually researchers hope to create a "general artificial intelligence" which can solve many problems instead of focusing on just one. Researchers are also trying to create creative and emotional AI which can possibly empathize or create art. Many approaches and tools have been tried.', metadata={'source': 'https://simple.wikipedia.org/wiki/Artificial_intelligence', 'title': 'Artificial intelligence - Simple English Wikipedia, the free encyclopedia', 'language': 'en'})

In [ ]:
chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             verbose = False)

In [ ]:
output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

Artificial intelligence (AI) is the ability of computers or machines to think and learn without
explicit programming. It encompasses various fields and aims to develop intelligent behavior in
machines. AI has been used successfully in areas such as speech recognition, game systems, self-
driving cars, and data interpretation. However, there are concerns about its potential dangers. AI
research began in the 1950s but faced challenges and funding cuts, leading to a decline known as the
"AI winter." It experienced a resurgence in the 90s and early 2000s, with notable achievements in
defeating human champions in chess and Jeopardy!. AI is a multidisciplinary field and researchers
aim to develop general and creative AI. The summary also mentions related pages, references, and the
need for global collaboration in navigating the AI landscape.


In [ ]:
prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""

In [ ]:
prompt = PromptTemplate.from_template(prompt_template)

In [ ]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="text"
  )

In [ ]:
output_summary = stuff_chain.run(docs)

In [ ]:
wrapped_text = textwrap.fill(output_summary,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

Artificial intelligence (AI) is the ability of a computer program or machine to think and learn. It
is a field of study that aims to make computers "smart" and capable of working on their own without
being explicitly programmed. The term "intelligence" in AI refers to the ability to act and make
decisions, rather than thinking like a human. AI has applications in various fields such as speech
recognition, strategic game systems, self-driving cars, and data interpretation. Researchers are
working towards creating a general artificial intelligence that can solve multiple problems and even
exhibit creativity and emotions. However, AI also raises concerns about its potential impact on
humanity.


In [ ]:
chain = load_summarize_chain(llm,
                             chain_type="stuff",
                             prompt=prompt)

## ⌛**Map Chain**

In [ ]:
map_template = """Write a concise summary of the following content:

{text}

Summary:
"""

In [ ]:
map_prompt = PromptTemplate.from_template(map_template)

In [ ]:
map_chain = LLMChain(prompt=map_prompt, llm=llm)

## ⌛**Reduce Chain**

In [ ]:
reduce_template = """The following is set of summaries:

{doc_summaries}

Summarize the above summaries with all the key details
Summary:"""

In [ ]:
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [ ]:
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

In [ ]:
stuff_chain = StuffDocumentsChain(llm_chain=reduce_chain,
                                  document_variable_name="doc_summaries")

In [ ]:
reduce_chain = ReduceDocumentsChain(
    combine_documents_chain=stuff_chain
)

## ⌛**Map Reduce Chain**

In [ ]:
map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    document_variable_name="text",
    reduce_documents_chain=reduce_chain)

In [ ]:
output = map_reduce_chain.run(docs)

In [ ]:
wrapped_text = textwrap.fill(output,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

Artificial intelligence (AI) is a field of computer science that focuses on creating intelligent
machines capable of tasks that typically require human intelligence. It is used in various
industries and has the potential to greatly impact society. The term "intelligence" is debated in
AI, with different perspectives on whether it should be defined in terms of action or thinking. AI
involves computer programs that mimic human cognition and can learn, solve problems, and think
logically. It is a multidisciplinary field that encompasses various disciplines. AI research began
in 1956 but experienced a decline known as the "AI winter" before resurging in the 90s and early
2000s. Notable achievements include machines defeating human champions in chess and Jeopardy! AI has
become popular worldwide due to advancements in technology and access to more data. The content also
includes references to various topics and sources discussing AI.


In [ ]:
chain = load_summarize_chain(llm, chain_type="refine")
chain.run(docs)

'Artificial intelligence (AI) is the ability of a computer program or machine to think and learn, mimicking human cognition. It involves the interpretation of external data, learning from it, and using those learnings to achieve specific goals and tasks through flexible adaptation. The goal of AI is to create a flexible agent that can perceive its environment and take actions to maximize its chance of success. AI involves many different fields like computer science, mathematics, linguistics, psychology, neuroscience, and philosophy. While the extreme goal of AI research is to create computer programs that can learn, solve problems, and think logically, in practice, most applications focus on tasks that computers can do well, such as searching databases and performing calculations. The ability for AI to truly perceive its environment is currently beyond the capabilities of present-day computing. Researchers are also exploring the creation of a "general artificial intelligence" that can 

Additionally, it's worth noting that Langchain offers the flexibility to provide a prompt and obtain intermediate steps in the summarization process. This feature enables a more detailed understanding of how the summarization is being generated, making it a valuable tool for transparency and fine-tuning the summarization outcomes.

In [ ]:
prompt_template = """Write a concise summary of the following extracting the key information:

{text}

CONCISE SUMMARY:"""

In [ ]:
prompt = PromptTemplate(template=prompt_template,
                        input_variables=["text"])

In [ ]:
refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary"
    "If the context isn't useful, return the original summary."
)

In [ ]:
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [ ]:
chain = load_summarize_chain(OpenAI(temperature=0),
                             chain_type="refine",
                             return_intermediate_steps=True,
                             question_prompt=prompt,
                             refine_prompt=refine_prompt)

In [ ]:
output_summary = chain({"input_documents": docs}, return_only_outputs=True)

In [ ]:
wrapped_text = textwrap.fill(output_summary['output_text'],
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)



Artificial intelligence (AI) is a field of study that focuses on creating machines that can think
and act like humans. It was first coined by John McCarthy in 1955 and is related to other fields
such as robotics, natural language processing, and machine learning. AI involves many different
fields like computer science, mathematics, linguistics, psychology, neuroscience, and philosophy. AI
is used in many areas, including healthcare, finance, and transportation, and is spoken in many
languages. The term intelligence is misleading, as Alan Turing wrote in 1950 that the question
should be changed from whether a machine "thinks" to "whether or not it is possible for machinery to
show intelligent behaviour". Turing also created the Turing test, which is a general test to
determine if a machine is intelligent. AI founders John McCarthy and Russell and Norvig agree that
AI must be defined in terms of "acting" and not "thinking", and that the Turing test should not
compare machines to people